## 1\Setup

In [95]:
%pip install openai
%pip install pandas
%pip install scikit-learn
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [96]:
import os
import ast
import pandas as pd
import numpy as np
import kagglehub
import random
from copy import deepcopy

from openai import OpenAI
import os
import re
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import json
import matplotlib

In [97]:
datagen_model = "gpt-4o"
api_key = '<yourkey>' 
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", api_key))

BatchSize = 5 # For each gpt request, access 3 misconceptions

## 2\MisconceptionName Mapping

In [98]:
data_dir = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"

df = pd.read_csv(os.path.join(data_dir, "train.csv"))
content_df = pd.read_csv(os.path.join(data_dir, "misconception_mapping.csv"))

content_df["MisconceptionId"] = content_df["MisconceptionId"].astype(str)
id2name = dict(zip(content_df["MisconceptionId"], content_df["MisconceptionName"]))

In [99]:
misconception_ids = set()
for col in ["MisconceptionAId", "MisconceptionBId", "MisconceptionCId", "MisconceptionDId"]:
    misconception_ids.update(df[col].dropna().astype(int).unique())
misconception_ids = list(map(str, misconception_ids))
print(f"Number of unique MisconceptionIds: {len(misconception_ids)}")

Number of unique MisconceptionIds: 1604


In [100]:
missing_ids = content_df[~content_df["MisconceptionId"].isin(misconception_ids)]["MisconceptionId"].unique().tolist()
len(missing_ids)

983

In [101]:
all_ids = missing_ids + misconception_ids
len(all_ids)

2587

In [102]:
misconception_ids = set()
for col in ["MisconceptionAId", "MisconceptionBId", "MisconceptionCId", "MisconceptionDId"]:
    misconception_ids.update(df[col].dropna().astype(int).unique())
misconception_ids = list(map(str, misconception_ids))
print(f"Number of unique MisconceptionIds: {len(misconception_ids)}")

Number of unique MisconceptionIds: 1604


In [103]:
df = df.join(content_df,on='MisconceptionAId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionAName'
df = df.join(content_df,on='MisconceptionBId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionBName'
df = df.join(content_df,on='MisconceptionCId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionCName'
df = df.join(content_df,on='MisconceptionDId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionDName'

## 3\Functions

In [104]:
def gen_reference_mcq(qid_list,df):
    qlist = []
    for q in qid_list:
        SubjectName = df.loc[q,'SubjectName']
        ConstructName = df.loc[q,'ConstructName']
        QuestionText = df.loc[q,'QuestionText']
        AnswerAText = df.loc[q,'AnswerAText']
        AnswerBText = df.loc[q,'AnswerBText']
        AnswerCText = df.loc[q,'AnswerCText']
        AnswerDText = df.loc[q,'AnswerDText']
        A_ifcorrect = "correct" if df.loc[q, 'CorrectAnswer'] == 'A' else "incorrect"
        B_ifcorrect = "correct" if df.loc[q, 'CorrectAnswer'] == 'B' else "incorrect"
        C_ifcorrect = "correct" if df.loc[q, 'CorrectAnswer'] == 'C' else "incorrect"
        D_ifcorrect = "correct" if df.loc[q, 'CorrectAnswer'] == 'D' else "incorrect"
        MisconceptionA = "No matched misconception" if pd.isna(df.loc[q, 'MisconceptionAName']) else df.loc[q,'MisconceptionAName']
        MisconceptionB = "No matched misconception" if pd.isna(df.loc[q, 'MisconceptionBName']) else df.loc[q,'MisconceptionBName']
        MisconceptionC = "No matched misconception" if pd.isna(df.loc[q, 'MisconceptionCName']) else df.loc[q,'MisconceptionCName']
        MisconceptionD = "No matched misconception" if pd.isna(df.loc[q, 'MisconceptionDName']) else df.loc[q,'MisconceptionDName']
        MisconceptionAId = int(df.loc[q,'MisconceptionAId']) if not pd.isna(df.loc[q,'MisconceptionAId']) else -1 
        MisconceptionBId = int(df.loc[q,'MisconceptionBId']) if not pd.isna(df.loc[q,'MisconceptionBId']) else -1 
        MisconceptionCId = int(df.loc[q,'MisconceptionCId']) if not pd.isna(df.loc[q,'MisconceptionCId']) else -1 
        MisconceptionDId = int(df.loc[q,'MisconceptionDId']) if not pd.isna(df.loc[q,'MisconceptionDId']) else -1 
        reference_mcq = f''' Question {q}
Subject: {SubjectName}  Construct: {ConstructName}
Question: {QuestionText}
AnswerA: {AnswerAText}
The answer is: {A_ifcorrect}.MisconceptionA is {MisconceptionAId}: {MisconceptionA}
AnswerB: {AnswerBText}
The answer is: {B_ifcorrect}.MisconceptionB is {MisconceptionBId}: {MisconceptionB}
AnswerC: {AnswerCText}
The answer is: {C_ifcorrect}.MisconceptionC is {MisconceptionCId}: {MisconceptionC}
AnswerD: {AnswerDText}
The answer is: {D_ifcorrect}.MisconceptionD is {MisconceptionDId}: {MisconceptionD}
        '''
        qlist.append(reference_mcq)
    return '\n'.join(qlist)

In [105]:
def get_res(mids,rids,df): # mids is a list of missing misconceptionid, rids is a list of reference mid
    rows = []
    for i in mids:
        rows.append('Id:'+ content_df.loc[int(i),'MisconceptionId'] + ' Misconception:' +content_df.loc[int(i),'MisconceptionName'])
    cluster_misconceptions = '\n'.join(rows)
    reference_mcqs = gen_reference_mcq(rids,df) #input
    num_mcqs = len(mids) # generate 2 questions for each misconception
    
    prompt = f"""
    You will be generating Multiple Choice Questions (MCQs) that diagnose specific mathematical misconceptions. Here are the misconceptions you should focus on:
    
    <misconceptions>
    {cluster_misconceptions}
    </misconceptions>
    
    Here are reference MCQs that demonstrate the questions and misconceptions:
    
    <reference_mcqs>
    {reference_mcqs}
    </reference_mcqs>
    
    Your task is to generate {num_mcqs} new MCQs that diagnose misconceptions not already covered by the reference MCQs.
    
    First, analyze the reference MCQs carefully: 
    1. For each reference MCQ, identify in your <analysis> tags:
       - Which misconception it targets
       - Try to solve the question and understand how the incorrect answers map to specific misconceptions
    2. Note the style, difficulty level, and precision of language used
    3. Note that incorrect answers do not necessartily correspond to a misconception, sometimes they are just random wrong answers
    
    Then, in your <planning> tag:
    - List which misconceptions still need coverage
    - For each needed misconception, brainstorm mathematical contexts where it commonly appears
    - Design questions where the misconception leads to specific wrong answers
    - Take notes on how you can craft new MCQs that adheres to the reference MCQs' style, difficulty level, and precision of language
    
    Finally, generate new MCQs following these important guidelines:
    - For each question, before giving answers, add a <reasoning> tag, you first solve the probelm yourself and then analyze the answer the misconconception leads to
    - Then generate MCQs according to your reasoning, ensure the answers are plausible and stem from genuine misconceptions, not careless errors
    - For each question, make sure at least one of MiconceptionA,B,C,D corresponds to one of <misconceptions>, you're allowed to have unmatched misconceptions for other incorrect answers.
    - For each question, the correct answer can be AnswerA,B,C,D randomly, don't always set up AnswerA as the correct answer
    - Only design one incorrect answer that 
    - Make questions challenging enough that students must demonstrate real understanding
    - Make sure AnswerA,B,C,D are different from each other, and there is only one correct answer among A,B,C,D
    - Use precise mathematical language matching the style of reference MCQs
    - Keep the construct name and subject name as short as possible
    
    """
    response = client.chat.completions.create(
    model=datagen_model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to generate synthetic data."},
        {"role": "user", "content": prompt}
      ]
    )
    res = response.choices[0].message.content
    return res


In [116]:
def extract_data(res):
    constructs = re.findall(r"Construct:\s*(.*?)\n", res)
    subjects = re.findall(r"Subject:\s*(.*?)\n", res)
    correct_answers = re.findall(r"Answer([A-D]):.*?\nThe answer is: correct", res)
    questions = re.findall(r"Question:\s*(.*?)\n", res)
    AnswerA = re.findall(r"AnswerA:(.*?)\n", res)
    AnswerB = re.findall(r"AnswerB:(.*?)\n", res)
    AnswerC = re.findall(r"AnswerC:(.*?)\n", res)    # Create a DataFrame
    AnswerD = re.findall(r"AnswerD:(.*?)\n", res)
    MisconceptionA = re.findall(r"MisconceptionA is *(.*?):", res)
    MisconceptionB = re.findall(r"MisconceptionB is *(.*?):", res)
    MisconceptionC = re.findall(r"MisconceptionC is *(.*?):", res)    # Create a DataFrame
    MisconceptionD = re.findall(r"MisconceptionD is *(.*?):", res)
    df = pd.DataFrame({
        'ConstructId': constructs,
        'SubjectName': subjects,
        'CorrectAnswer': correct_answers,
        'QuestionText': questions,
        'AnswerAText': AnswerA,
        'AnswerBText': AnswerB,
        'AnswerCText': AnswerC,
        'AnswerDText': AnswerD,
        'MisconceptionAId': MisconceptionA,
        'MisconceptionBId': MisconceptionB,
        'MisconceptionCId': MisconceptionC,
        'MisconceptionDId': MisconceptionD,
    })
    return df

## 4\ synthesize data

In [117]:
i = 0
res_list = []
df_list = []
misconception_ids = [int(x) for x in misconception_ids]
missing_ids = [int(x) for x in missing_ids]

while i<=len(missing_ids)//BatchSize:
    mids = missing_ids[BatchSize*i:BatchSize*(i+1)] if BatchSize*(i+1)<=len(missing_ids) else len(missing_ids)
    rids = misconception_ids[BatchSize*i:BatchSize*(i+1)]
    res = get_res(mids,rids,df)
    res_list.append(res)
    syn_df = extract_data(res)
    df_list.append(syn_df)
    i += BatchSize
    # break
syn_dfs = pd.concat(df_list)

In [138]:
content_df['MisconceptionId']=content_df['MisconceptionId'].astype('int64')
syn_dfs['MisconceptionAId']=syn_dfs['MisconceptionAId'].astype('int64')
syn_dfs['MisconceptionBId']=syn_dfs['MisconceptionBId'].astype('int64')
syn_dfs['MisconceptionCId']=syn_dfs['MisconceptionCId'].astype('int64')
syn_dfs['MisconceptionDId']=syn_dfs['MisconceptionDId'].astype('int64')

In [139]:
syn_dfs = syn_dfs.join(content_df,on='MisconceptionAId')
syn_dfs = syn_dfs.drop('MisconceptionId',axis=1)
syn_dfs.columns.values[-1] = 'MisconceptionAName'
syn_dfs = syn_dfs.join(content_df,on='MisconceptionBId')
syn_dfs = syn_dfs.drop('MisconceptionId',axis=1)
syn_dfs.columns.values[-1] = 'MisconceptionBName'
syn_dfs = syn_dfs.join(content_df,on='MisconceptionCId')
syn_dfs = syn_dfs.drop('MisconceptionId',axis=1)
syn_dfs.columns.values[-1] = 'MisconceptionCName'
syn_dfs = syn_dfs.join(content_df,on='MisconceptionDId')
syn_dfs = syn_dfs.drop('MisconceptionId',axis=1)
syn_dfs.columns.values[-1] = 'MisconceptionDName'

In [141]:
syn_dfs.to_csv('synthesized_data_gpt_4o.csv')